LCEL (LangChain Expression Language) is a domain-specific language in LangChain designed to simplify how developers create and manipulate prompts, chains, and outputs. It allows combining LLM calls, variables, and logic in a concise, readable syntax without writing full Python code. LCEL is useful for building complex workflows quickly, like conditional flows or multi-step reasoning, while keeping everything declarative. It improves maintainability by separating the chain logic from the code itself. Essentially, LCEL makes LangChain projects more modular, flexible, and easier to reason about.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7) 
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020A43C4E6C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020A43D9A210>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage , SystemMessage
from sympy import content
messages = [
    SystemMessage(content="translate from english to french"),
    HumanMessage(content="hello how are you")
]

result = llm.invoke(messages)

this will give response as well as metadata

In [5]:
result 

AIMessage(content='Bonjour, comment allez-vous ? \n\n(Translation: Hello, how are you?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 44, 'total_tokens': 62, 'completion_time': 0.048611798, 'prompt_time': 0.011297178, 'queue_time': 0.051036362, 'total_time': 0.059908976}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c1fab78-2678-4e2f-bc85-156a432f181b-0', usage_metadata={'input_tokens': 44, 'output_tokens': 18, 'total_tokens': 62})

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n\n(Translation: Hello, how are you?)'

chaining a prompt template with the LLaMA-3.3-70B model and an output parser to process the response (e.g., "Bonjour, comment vas-tu ?") into a structured format.

In [7]:
# using lcel - chain the components
chain = llm|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [8]:
#prompt template - convert into list of message 
from langchain_core.prompts import ChatPromptTemplate

template = "translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system",template),("user","{text}")]
)

In [9]:
result = prompt.invoke({"language":"french","text":"hello"})
result

ChatPromptValue(messages=[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

In [10]:
result.to_messages()

[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [11]:
chain=prompt|llm|parser
chain.invoke({"language":"french","text":"hello"})

'Bonjour'